**Importing .csv file**

In [1]:
import pandas as pd
import re

from sklearn.metrics import accuracy_score, classification_report 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost
import lightgbm
import catboost

**Importing tweets data**

In [2]:
df_tweets = pd.read_csv('../data/Tweets.csv', usecols=['text', 'selected_text', 'sentiment'])
pd.set_option('display.max_columns', None)

df_tweets.head()

,text,selected_text,sentiment
0,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,my boss is bullying me...,bullying me,negative
3,what interview! leave me alone,leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [3]:
df_tweets.sentiment.value_counts()

neutral     11118
positive     8582
negative     7781
Name: sentiment, dtype: int64

**Removing records with null values**

In [26]:
df_tweets = df_tweets.drop(df_tweets[df_tweets['text'].isna()].index.tolist())
df_tweets.reset_index(drop=True, inplace=True)
print(df_tweets.isna().sum())

text             0
selected_text    0
sentiment        0
dtype: int64


In [27]:
df_tweets.head()

,text,selected_text,sentiment
0,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,my boss is bullying me...,bullying me,negative
3,what interview! leave me alone,leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


**Cleaning up the text**

In [30]:
def clean_text(df, col_name):

    processed_features = []

    for sentence in range(0, len(df)):
        # Remove all the special characters
        processed_feature = re.sub(r'\W', ' ', str(df.loc[sentence, 'text']))

        # remove all single characters
        processed_feature = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

        # Remove single characters from the start
        processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 

        # Substituting multiple spaces with single space
        processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

        # Removing prefixed 'b'
        processed_feature = re.sub(r'^b\s+', '', processed_feature)

        # Converting to Lowercase
        processed_feature = processed_feature.lower().strip()

        processed_features.append(processed_feature)

    return processed_features

In [31]:
df_tweets['text'] = clean_text(df_tweets, 'text')
df_tweets['text'].head()

0                       d have responded if were going
1             sooo sad will miss you here in san diego
2                               my boss is bullying me
3                        what interview leave me alone
4    sons of why couldn they put them on the releas...
Name: text, dtype: object

**Creating train, val, and test datasets** 

In [32]:
df_fulltrain, df_test = train_test_split(df_tweets[['text', 'sentiment']], test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_fulltrain, test_size=0.25, random_state=1)

len(df_train), len(df_val), len(df_test), len(df_tweets)

(16488, 5496, 5496, 27480)

In [33]:
df_train.shape, df_val.shape, df_test.shape

((16488, 2), (5496, 2), (5496, 2))

In [34]:
df_train.columns

Index(['text', 'sentiment'], dtype='object')

In [35]:
y_train = df_train.sentiment.values
y_val = df_val.sentiment.values
y_test = df_test.sentiment.values

del df_train['sentiment']
del df_val['sentiment']
del df_test['sentiment']

In [36]:
df_train.head()

,text
20135,listening to fountain of youth by supastition ...
12611,no sir they did not was amazed when woke up th...
10342,awww that sucks but they re so awesome when yo...
12499,leno last show tonight
8661,thanks funny cause it true


**Converting string to bag of words**

In [37]:
tfidf = TfidfVectorizer()
X_train = tfidf.fit_transform(df_train['text']).toarray()
X_val = tfidf.transform(df_val['text']).toarray()

In [38]:
X_train.shape

(16488, 19278)

**Training Random Forests model**

This ensemble algorithm (i.e. uses multiple ML models) fits multiple decision tree classifiers on the dataset, and picks the majority vote among all decision trees as the final ouptut of the model. Some important hyperparameters of this model are, n_estimators (number of trees), bootstrap (use subsamples to build each tree instead of all data), max_depth (defines the maximum depth of all trees), etc. 

In [39]:
model = RandomForestClassifier(random_state=1)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

**Evaluating model**

In [40]:
y_pred = model.predict(X_val)

In [41]:
print(accuracy_score(y_val, y_pred))

0.6743085880640466


In [42]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

    negative       0.72      0.51      0.60      1538
     neutral       0.61      0.78      0.68      2269
    positive       0.77      0.68      0.72      1689

    accuracy                           0.67      5496
   macro avg       0.70      0.66      0.67      5496
weighted avg       0.69      0.67      0.67      5496



**Training SVM model**

SVMs create a decision boundary, which is a plane or line, that can seperate data clusters with the largest margin (which is the sum of the distances from the decision boundary to the nearest datapoint in both data clusters being classified) possible. If the data is not linearly seperable, SVMs extend the data into a higher dimension, by using functions known as kernels, and then finds a decision boundary in this new dimension. Kernels apply certain transformations on the data, which generate new features that are used to extend the dimension of the data, thereby hopefully making them linearly seperable. For example, the polynomial kernel takes one or more of the features and raises them to some power greater than or equal to 2.

In [43]:
model_SVM = SVC(random_state=1)
model_SVM.fit(X_train, y_train)

**Evaluation**

**Training XGBoost model**

Uses gradient boosting, which is a ensemble algorithm that combines multiple decision trees in sequential order, where each subsequent tree is focused on better predicting the cases where the previous tree performed poorly. This process is repeated multiple times, which results in better predictions than if a single model was used.  

In [ ]:
xgboost()

**Evaluation**

**Training LightGBM model**

A gradient boosting algorithm similar to XGBoost. However, LightGBM is significantly faster at training than XGBoost, due to use of leaf-wise growth of decision trees, as compared to XGBoost's level-wise growth. This results in higher accuracy, but could also cause overfitting.

**Evaluation**

**Training CatBoost model**

Another gradient boosting algorithm. CatBoost contains native support for many feature types, such as numerical, categroical, and text. Additionally, CatBoost uses symmetrical decision trees, which allows for faster predictions, and reduces the chances of overfitting.

**Evaluation**

**Dumping model and tfid vectorizer to an external file**

In [ ]:
import pickle

In [ ]:
output_file = "../models/model.bin"

f_out = open(output_file, 'wb') 
pickle.dump((tfidf, model), f_out)
f_out.close()